In [1]:
from student.agent.agent_raspa import *
from student.agent.tools.tools_memory import *
from agent.student import *
from student.agent.utils import *
from student.agent.agent_memory import *

In [2]:
import mllm
mllm.config.default_models.expensive = "claude-sonnet-4-20250514"

# Test Student

In [3]:
a = StudentAgent(provider="anthropic")

In [4]:
a.run("my name is henrik")
#a.run("what is my name")
a.run("my name is peter")
a.render_conversation()

# Test tools, memory

In [31]:
a = Agent()


In [9]:
path = "output/st"
r = RaspaAgent(path, version="v2.xml")

In [12]:
r.tools['trappe'].run(["Methane"])

'<tool response name=molecule loader>\n            Successfully generated the molecule input files (and force field files) for: \n            <file name=Methane.def></file>\n            </tool response>'

In [ ]:
r.run("My name is Henrik and my father is called Stefan")
r.render_chat_html()

In [ ]:
r.reset_chat()
r.run("My name is Henrik and my father is called Stefan")
r.render_chat_html()

## Finished tests

In [ ]:
#r.run("which files can you see in the current directory? Try to access the propane related file")
#r.render_chat_html()

In [ ]:
a='''Test read/write: works as expected!'''
r.run("write a file containing a short list of random words")
#r.run("what is contained in the file random_words.txt?")
r.render_chat_html()

In [ ]:
a='''Be careful with old memory implementation:
for i in r.tools['trappe'].memory.search(['nonsense']):
    print(i.content)
'''

In [ ]:
a='''
r = RaspaAgent(path)
r.run("generate the file for ABAVIJ_clean")
r.run("It is the name of a MOF")
r.render_chat_html()
'''

a='''File generation works!
r.run("Generate the molecule input files for carbon dioxide and propane")
r.render_chat_html()
'''

In [10]:
r.reset(path="output/raspa_test")
r.run("create a molecule file for propane")

'The molecule file for propane has already been created.'

In [ ]:
r.run("create a framework file for ABAVIJ")

# Test the tools

In [ ]:
agent = MemoryAgent()
tools = agent.tools
add = tools['add'].run
recall = tools['recall'].run
modify = tools['modify'].run
memory = agent.memory

In [3]:
add(["chemistry"], "Chemistry is a discipline that is linked to <molecules/> and <chemical bonds/>")
add(["chemistry", "molecules"], "Molecules are <atoms/> connected by <chemical bonds/>")
add(["chemistry", "atoms"], "Each atom has one fixed type which can be any from the <periodic table/>")
add(["chemistry", "chemical bonds"], "A chemical bond is really complicated and not really understood yet. There are many different models, from <quantum mechanics/> to more <classical bonding models/> that provide a rich theory for predicting chemical reactions.")
add(["chemistry", "chemical bonds", "quantum mechanics"], "This is really wild <physics/> stuff.")
add(["chemistry", "periodic table"], "The table summarizes all types an <atom/> can have.")
add(["physics"], "Physics is just applied math.")
memory.render_html()

In [4]:
print(recall(["chemistry"], sensitivity=0)) # not good!

<tool response name=recall>Recalled: 
	
        <memory id="f616ddb7">
            <stimuli>chemistry</stimuli>
            <content>Chemistry is a discipline that is linked to <molecules/> and <chemical bonds/></content>
        </memory>
        
        <memory id="bf757d23">
            <stimuli>molecules, chemistry</stimuli>
            <content>Molecules are <atoms/> connected by <chemical bonds/></content>
        </memory>
        
        <memory id="7d855cdf">
            <stimuli>chemical bonds, chemistry</stim</tool response>


In [5]:
import numpy as np
sensitivity = 0.2
max_recall = 5
queries = ["chemistry"]

self = memory
scores = []
nodes = self.get_nodes()

for node in nodes:
    node_scores = node.get_score(queries, sensitivity=sensitivity)
    node_scores = np.array(node_scores)

    scores.append(node_scores)

scores = np.array(scores) # m
#score_summation_for_src = np.sum(scores, axis=0)

#score_norm_factor_for_src = score_summation_for_src + (score_summation_for_src == 0.0)
#scores = scores / score_norm_factor_for_src
#scores = np.sum(scores, axis=1)

excited_nodes = {}

top_k_indices = np.argsort(-scores)

for i in range(min(max_recall, len(top_k_indices))):
    s = scores[top_k_indices[i]]
    if s <= sensitivity:
        break
    node : MemoryNode = nodes[top_k_indices[i]]
    excited_nodes[node.id] = (s, node.keys)#node.__str__()
excited_nodes


{'f616ddb7': (1.0000001, {'chemistry'}),
 'bf757d23': (0.732136, {'chemistry', 'molecules'}),
 '7d855cdf': (0.7196873, {'chemical bonds', 'chemistry'}),
 '2357504c': (0.71602017, {'atoms', 'chemistry'}),
 '08fa7e25': (0.68705606, {'chemistry', 'periodic table'})}

# Test Memory agent: easy example

In [31]:
ag = StudentAgent(version="v2.xml")

In [32]:
ag.run("colour_function(x) is a function that maps an integer x to a color. x is the index of the color sequence of the random-bow. The index starts at 4 and all invalid values return the color gray.")
ag.render_chat_html()

In [26]:
ag.reset_chat()
ag.run("The random-bow has 7 colors: red, blue, orange, green, violet, pink, green, yellow")
ag.render_chat_html()

In [27]:
ag.reset_chat()
ag.run("What is the value of the mapping colour_function(1). Use a low sensitivity")
ag.render_chat_html()

In [30]:
ag.chat.messages[-6]

{'role': 'user',
 'content': {'type': 'text',
  'text': '{"tool_call_id": 3, "tool_response": "only integer scalar arrays can be converted to a scalar index"}'}}

In [8]:
ag.reset_chat()
ag.run("What is the value of the mapping colour_function(6). Use a low sensitivity")
ag.render_chat_html()

## Math example

In [ ]:
ag.run("What do you do with new knowledge?")
ag.render_chat_html()

In [ ]:
"""
import mllm
c = mllm.Chat(system_message="You are an intelligent agent that uses step by step reasoning to solve questions.")
prompt='''
To determine a extreme point(minimum,maximum,saddlepoint) of a mathematical function, you need to determine the derivative and set it equal to zero.
The derivative of x^2 is 2x. The derivative of ln(x) is 1/x.
The derivative of a product of functions f(x)*g(x) can be calculated with this formula: derivative[f(x)]*g(x) + f(x)*derivative[g(x)]
What is the minimum of (ln(x))^2?
'''
c += prompt
res = c.complete()
print(res)
#"To find the minimum of the function \\( f(x) = (\\ln(x))^2 \\), we will follow these steps:\n\n1. **Find the derivative of the function**.\n2. **Set the derivative equal to zero to find critical points**.\n3. **Determine whether these critical points correspond to a minimum, maximum, or saddle point**.\n\n### Step 1: Find the derivative\n\nUsing the chain rule, the derivative of \\( f(x) = (\\ln(x))^2 \\) can be calculated as follows:\n\n\\[\nf'(x) = 2 \\ln(x) \\cdot \\frac{d}{dx}(\\ln(x)) = 2 \\ln(x) \\cdot \\frac{1}{x} = \\frac{2 \\ln(x)}{x}\n\\]\n\n### Step 2: Set the derivative equal to zero\n\nNow we set the derivative equal to zero to find the critical points:\n\n\\[\n\\frac{2 \\ln(x)}{x} = 0\n\\]\n\nThis equation is satisfied when \\( \\ln(x) = 0 \\). Solving for \\( x \\):\n\n\\[\n\\ln(x) = 0 \\implies x = e^0 = 1\n\\]\n\n### Step 3: Determine the nature of the critical point\n\nTo determine whether this critical point is a minimum, maximum, or saddle point, we can use the second derivative test. We first find the second derivative \\( f''(x) \\).\n\nStarting from \\( f'(x) = \\frac{2 \\ln(x)}{x} \\), we will differentiate it again using the quotient rule:\n\nLet \\( u = 2 \\ln(x) \\) and \\( v = x \\). Then:\n\n\\[\nf''(x) = \\frac{u'v - uv'}{v^2}\n\\]\n\nCalculating \\( u' \\) and \\( v' \\):\n\n- \\( u' = \\frac{2}{x} \\)\n- \\( v' = 1 \\)\n\nNow substituting into the quotient rule:\n\n\\[\nf''(x) = \\frac{\\left(\\frac{2}{x}\\right)x - (2 \\ln(x))(1)}{x^2} = \\frac{2 - 2 \\ln(x)}{x^2}\n\\]\n\nNow we evaluate the second derivative at the critical point \\( x = 1 \\):\n\n\\[\nf''(1) = \\frac{2 - 2 \\ln(1)}{1^2} = \\frac{2 - 2 \\cdot 0}{1} = 2\n\\]\n\nSince \\( f''(1) > 0 \\), this indicates that the function \\( f(x) \\) has a local minimum at \\( x = 1 \\).\n\n### Conclusion\n\nThe minimum value of \\( f(x) = (\\ln(x))^2 \\) occurs at \\( x = 1 \\). To find the minimum value:\n\n\\[\nf(1) = (\\ln(1))^2 = 0^2 = 0\n\\]\n\nThus, the minimum of \\( (\\ln(x))^2 \\) is \\( 0 \\) at \\( x = 1 \\)."
"""

'\nimport mllm\nc = mllm.Chat(system_message="You are an intelligent agent that uses step by step reasoning to solve questions.")\nprompt=\'\'\'\nTo determine a extreme point(minimum,maximum,saddlepoint) of a mathematical function, you need to determine the derivative and set it equal to zero.\nThe derivative of x^2 is 2x. The derivative of ln(x) is 1/x.\nThe derivative of a product of functions f(x)*g(x) can be calculated with this formula: derivative[f(x)]*g(x) + f(x)*derivative[g(x)]\nWhat is the minimum of (ln(x))^2?\n\'\'\'\nc += prompt\nres = c.complete()\nprint(res)\n#"To find the minimum of the function \\( f(x) = (\\ln(x))^2 \\), we will follow these steps:\n\n1. **Find the derivative of the function**.\n2. **Set the derivative equal to zero to find critical points**.\n3. **Determine whether these critical points correspond to a minimum, maximum, or saddle point**.\n\n### Step 1: Find the derivative\n\nUsing the chain rule, the derivative of \\( f(x) = (\\ln(x))^2 \\) can be c

In [9]:
ag = StudentAgent(version="v2.xml")
res = ag.run("To determine a extreme point(minimum,maximum,saddlepoint) of a mathematical function, you need to determine the derivative and set it equal to zero.")
res = ag.run("The derivative of x^2 is 2x. The derivative of ln(x) is 1/x.")
res = ag.run("The derivative of a product of functions f(x)*g(x) can be calculated with this formula: derivative[f(x)]*g(x) + f(x)*derivative[g(x)]")
res = ag.run("What is the minimum of (ln(x))^2")
ag.render_chat_html()

# Memory Agent Test Runs

In [ ]:
ag = StudentAgent()
ag.run("Do not do a test run of the add memory tool")
ag.run("use the add memory tool once for a test run with the key 'test'")
ag.run("use the recall memory tool for a test run with the key 'test'")
ag.render_chat_html()

In [ ]:
ag = StudentAgent()
ag.run("The secret password is zurich7891")
ag.run("Three elefants tried to access the club but did not know the password to enter and left again.")
ag.run("I want to access the club. What is the password?")
#ag.reset_chat()
ag.run("I want to access the club. What is the secret password? I told you earlier")
ag.render_chat_html()